In [1]:
cd ..

/home/is/akiyoshi-n/my-project


In [2]:
import os
# 使用するGPUを指定. この環境変数の場所は，pytorchをimportする前に入れる
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
from pathlib import Path
from datetime import datetime
from src.my_project.dataset import load_dataset_2class_classification,load_dataset_4class_Multi_classification, split_multilabel_data, split_data_by_index, load_dataset_3class_Multi_classification, split_same_texts_data
from src.my_project.train_v2 import ActivityPrediction
from sklearn.model_selection import train_test_split
from src.my_project.dataset import load_multiclass_dataset
import wandb
import numpy as np

In [3]:
DATASET_PATH = Path('/home/is/akiyoshi-n/my-project/data')
# 本日の日付
timestamp = datetime.now().strftime("%Y-%m-%d")
# 出力先ディレクトリ
output_dir = Path('/home/is/akiyoshi-n/my-project/outputs/{}'.format(timestamp))

In [4]:
# 最大トークン数
MAX_LEN = 128
# バッチサイズ
BATCH_SIZE = 16
# エポック数
NUM_EPOCHS = 100
# 学習率
LEARNING_RATE = 2e-5
# Cross Validation時のFold数
NUM_FOLDS = 3
# 早期停止のための忍耐値
PATIENCE = 5
# 乱数シード
SEED = 2023
# クラス数
NUM_LABELS = 4
# 閾値
THRESH = 0.5

In [10]:
# データの読み込み
single_data = load_dataset_2class_classification(f"{DATASET_PATH}/act_classification_final.xlsx")

In [6]:
# 訓練データと評価データを辞書型で抽出
single_train_dataset = {
    'texts': [single_data['texts'][i] for i in range(900)],
    'labels': [single_data['labels'][i] for i in range(900)]
}
single_eval_dataset = {
    'texts': [single_data['texts'][i] for i in range(900, 1100)],
    'labels': [single_data['labels'][i] for i in range(900, 1100)]
}

In [7]:
# データの読み込み
multi_data, class_name = load_dataset_4class_Multi_classification(f"{DATASET_PATH}/act_classification_final.xlsx")

In [8]:
# 訓練データと評価データを辞書型で抽出
multi_train_dataset = {
    'texts': [multi_data['texts'][i] for i in range(900)],
    'labels': [multi_data['labels'][i] for i in range(900)]
}
multi_eval_dataset = {
    'texts': [multi_data['texts'][i] for i in range(900, 1100)],
    'labels': [multi_data['labels'][i] for i in range(900, 1100)]
}

In [10]:
# 東北大BERT-v3
MODEL_NAME = 'cl-tohoku/bert-base-japanese-v3'
Classifier_model = ActivityPrediction(model_name = MODEL_NAME, seed=SEED, num_labels=4)

In [ ]:
prediction = Classifier_model.train_and_predict(single_train_dataset, single_eval_dataset, multi_train_dataset, multi_eval_dataset, single_eval_dataset, multi_eval_dataset, MAX_LEN, NUM_EPOCHS, LEARNING_RATE, BATCH_SIZE, PATIENCE, output_dir, project_name='2step')

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

def calculate_accuracy_f1(y_true, y_pred):
    # 各サンプルに対してAccuracyを計算
    accuracy = accuracy_score(y_true, y_pred)

    # 各サンプルに対してF1値を計算（マルチクラスの場合は'average'パラメータを適切に設定する）
    f1 = f1_score(y_true, y_pred, average='macro')

    return accuracy, f1

In [ ]:
a, b = calculate_accuracy_f1(multi_eval_dataset['labels'], prediction)

In [ ]:
print(a)
print(b)

### Cross validation

In [5]:
single_raw_data = load_dataset_2class_classification(f"{DATASET_PATH}/act_classification_final_ChatGPT4.xlsx")

In [6]:
# データの読み込み
multi_data, class_name = load_dataset_4class_Multi_classification(f"{DATASET_PATH}/act_classification_final_ChatGPT4.xlsx")

In [7]:
# testデータと訓練に使用するデータに分割
multi_data, multi_test_data = split_multilabel_data(data=multi_data, test_size=0.1, SEED=SEED)

In [8]:
dataset_labels_np = np.array(multi_data['labels'])
test_data_labels_np = np.array(multi_test_data['labels'])
print(dataset_labels_np.sum(axis=0))
print(test_data_labels_np.sum(axis=0))

[110 130 322 550]
[12 15 36 61]


In [9]:
single_data = split_same_texts_data(multi_data, single_raw_data)
single_test_data = split_same_texts_data(multi_test_data, single_raw_data)

In [10]:
# 東北大BERT-v3
MODEL_NAME = 'cl-tohoku/bert-base-japanese-v3'
Classifier_model = ActivityPrediction(model_name = MODEL_NAME, seed=SEED, num_labels=4)

In [11]:
# Cross Validation
prediction = Classifier_model.cross_validation(single_data, multi_data, single_test_data, multi_test_data, MAX_LEN, NUM_EPOCHS, LEARNING_RATE, BATCH_SIZE, PATIENCE, NUM_FOLDS, output_dir, project_name='2step_cross_validation')

-----------------Fold: 1-----------------


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cl-tohoku/bert-base-japanese-v3 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Parameter 'fn_kwargs'={'tokenizer': BertJapaneseTokenizer(name_or_path='cl-tohoku/bert-base-japanese-v3', vocab_size=32768, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False,

Map:   0%|          | 0/719 [00:00<?, ? examples/s]

Map:   0%|          | 0/360 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.696700,0.681843,0.583333,0.569735
2,0.626900,0.598404,0.675000,0.674696
3,0.535200,0.562506,0.697222,0.693243
4,0.371000,0.604345,0.708333,0.707681
5,0.248200,0.709756,0.719444,0.711701
6,0.136200,0.944571,0.708333,0.708313
7,0.043900,1.341443,0.702778,0.693992
8,0.032200,1.500210,0.700000,0.692561


...

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


Epoch,Training Loss,Validation Loss,Accuracy,Macro F1,Class F1,Class Recall,Class Precision
1,0.656000,0.635794,0.108571,0.184976,"[0.051, 0.297, 0.206]","[0.027, 0.256, 0.12]","[0.5, 0.355, 0.722]"
2,0.594200,0.598370,0.314286,0.218661,"[0.053, 0.0, 0.603]","[0.027, 0.0, 0.5]","[1.0, 0.0, 0.761]"
3,0.531100,0.550405,0.428571,0.448437,"[0.367, 0.308, 0.67]","[0.243, 0.233, 0.574]","[0.75, 0.455, 0.805]"
4,0.420000,0.479596,0.611429,0.678350,"[0.632, 0.622, 0.781]","[0.649, 0.651, 0.694]","[0.615, 0.596, 0.893]"
5,0.289500,0.509355,0.657143,0.653002,"[0.526, 0.638, 0.794]","[0.405, 0.698, 0.787]","[0.75, 0.588, 0.802]"
6,0.182000,0.424862,0.708571,0.729895,"[0.667, 0.695, 0.828]","[0.622, 0.767, 0.759]","[0.719, 0.635, 0.911]"
7,0.102500,0.452295,0.731429,0.746785,"[0.675, 0.731, 0.834]","[0.73, 0.791, 0.769]","[0.628, 0.68, 0.912]"
8,0.068300,0.538843,0.708571,0.679863,"[0.583, 0.643, 0.813]","[0.568, 0.628, 0.787]","[0.6, 0.659, 0.842]"
9,0.051000,0.628734,0.691429,0.685475,"[0.557, 0.68, 0.819]","[0.459, 0.791, 0.796]","[0.708, 0.596, 0.843]"
10,0.035800,0.578184,0.731429,0.710862,"[0.575, 0.699, 0.858]","[0.568, 0.674, 0.87]","[0.583, 0.725, 0.847]"


Map:   0%|          | 0/121 [00:00<?, ? examples/s]

Map:   0%|          | 0/74 [00:00<?, ? examples/s]

{'accuracy': 0.4782608695652174, 'macro_f1': 0.45295908886710773, 'class_f1': [0.381, 0.437, 0.39, 0.604], 'class_recall': [0.364, 0.467, 0.455, 0.542], 'class_precision': [0.4, 0.412, 0.341, 0.681]}


In [12]:
print(prediction)

[{'accuracy': 0.5583333333333333, 'macro_f1': 0.4798538884117304, 'class_f1': [0.353, 0.357, 0.532, 0.678], 'class_recall': [0.25, 0.333, 0.6, 0.672], 'class_precision': [0.6, 0.385, 0.477, 0.683]}, {'accuracy': 0.46153846153846156, 'macro_f1': 0.44355170184784787, 'class_f1': [0.353, 0.414, 0.48, 0.527], 'class_recall': [0.3, 0.4, 0.667, 0.407], 'class_precision': [0.429, 0.429, 0.375, 0.75]}, {'accuracy': 0.4782608695652174, 'macro_f1': 0.45295908886710773, 'class_f1': [0.381, 0.437, 0.39, 0.604], 'class_recall': [0.364, 0.467, 0.455, 0.542], 'class_precision': [0.4, 0.412, 0.341, 0.681]}]


In [14]:
# 重みありの場合の結果（cv=5）解答
average_accuracy = round(sum(d['accuracy'] for d in prediction)/len(prediction), 3)
average_macro_f1 = round(sum(d['macro_f1'] for d in prediction)/len(prediction), 3)
# クラスごとの平均値を計算
average_class_f1 = [round(sum(d['class_f1'][i] for d in prediction) / len(prediction), 3) for i in range(len(prediction[0]['class_f1']))]
average_class_recall = [round(sum(d['class_recall'][i] for d in prediction) / len(prediction), 3) for i in range(len(prediction[0]['class_recall']))]
average_class_precision = [round(sum(d['class_precision'][i] for d in prediction) / len(prediction), 3) for i in range(len(prediction[0]['class_precision']))]
print("Average accuracy:", average_accuracy)
print("Average Macro f1:", average_macro_f1)
print("Average Class f1:", average_class_f1)
print("Average Class recall:", average_class_recall)
print("Average Class precision:", average_class_precision)

Average accuracy: 0.499
Average Macro f1: 0.459
Average Class f1: [0.362, 0.403, 0.467, 0.603]
Average Class recall: [0.305, 0.4, 0.574, 0.54]
Average Class precision: [0.476, 0.409, 0.398, 0.705]


### 一個前のコード変更する前の結果

In [ ]:
print(prediction)

In [ ]:
average_accuracy = sum(d['accuracy'] for d in prediction)/len(prediction)
average_f1 = sum(d['f1'] for d in prediction)/len(prediction)
print("Average accuracy:", average_accuracy)
print("Average f1:", average_f1)